In [1]:
import cv2
import numpy as np
import urllib.request as req
import tensorflow as tf

STRAWBERRY = 'strawberry'
LETTUCE = 'lettuce'
ROSEMARY = 'rosemary'
GERANIUM = 'geranium'

strawberry_dict = {
    'disease' : ['정상', '잿빛곰팡이병', '흰가루병', '해충'],
    'grow': ['1단계', '2단계', '3단계', '4단계', '5단계']
}
lettuce_dict = {
    'disease': ['정상', '균핵병', '노균병'],
    'grow': ['1단계', '2단계']
}
rosemary_dict = {
    'disease': ['정상', '흰가루병', '점무늬병', '해충']
}
geranium_dict = {
    'disease': ['정상', '갈색무늬병', '잿빛곰팡이병', '해충']
}

code_to_str = {
        STRAWBERRY : strawberry_dict, 
        LETTUCE: lettuce_dict, 
        ROSEMARY: rosemary_dict, 
        GERANIUM: geranium_dict}

def read_image_from_url(url, input_size = None) :
    resp = req.urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype='uint8')
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    if input_size is not None :
        image = cv2.resize(image, (input_size, input_size))
    return image / 255.

def load_plant_model(plant : str):
    if plant ==  STRAWBERRY :
        model = tf.keras.models.load_model('./strawberry_multi_output.h5')
    elif plant == LETTUCE :
        model = tf.keras.models.load_model('./lettuce_multi_output.h5')
    return model

def inference(url, plant:str) :
    model = load_plant_model(plant)
    if plant == STRAWBERRY or plant == LETTUCE :
        image = read_image_from_url(url, 320)
        image = np.array([image])
        disease, grow = model.predict(image)
        disease = np.argmax(disease[0])
        grow = np.argmax(grow[0])    
        return {'disease' : code_to_str[plant]['disease'][disease],
               'grow' : code_to_str[plant]['grow'][grow]}
    
    elif plant == ROSEMARY or plant == GERANIUM :
        image = read_image_from_url(url)
        image = np.array([image])
        disease = model.predict(image)
        disease = np.argmax(disease[0])
        return {'disease': code_to_str[plant]['disease'][disease]}

In [3]:
%%time
url = input()
inference(url, 'strawberry')

https://t1.daumcdn.net/cfile/tistory/99F53D335981A0221C


2022-06-19 00:34:55.452960: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-06-19 00:34:55.471677: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2499995000 Hz


CPU times: user 13.6 s, sys: 1.15 s, total: 14.8 s
Wall time: 15.7 s


{'disease': '잿빛곰팡이병', 'grow': '2단계'}